# MobileNet - Test

In [2]:
from tensorflow.keras import layers, models, applications
from tensorflow.keras.applications import mobilenet
from tensorflow.keras.utils import Sequence
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.utils import shuffle

import cv2

In [24]:
base_model = applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights=None
)

x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(1024, activation='relu', kernel_initializer='he_normal')(x)
x = layers.Dropout(0.3)(x)
output = layers.Dense(11, activation='softmax')(x)

model = models.Model(base_model.input, output)

In [5]:
# @title 데이터 train

# https://drive.google.com/file/d/1FvjGXjqWiI1R5k-Xror3R5CnRQXhfZAQ/view?usp=drive_link

import gdown, zipfile, os

file_id = '1FvjGXjqWiI1R5k-Xror3R5CnRQXhfZAQ'
output = 'file.zip'

gdown.download(f'https://drive.google.com/uc?id={file_id}', output, quiet=False)

output_dir = 'cloud_2'
os.makedirs(output_dir, exist_ok=True)

with zipfile.ZipFile(output, 'r') as z:
  z.extractall(output_dir)

Downloading...
From (original): https://drive.google.com/uc?id=1FvjGXjqWiI1R5k-Xror3R5CnRQXhfZAQ
From (redirected): https://drive.google.com/uc?id=1FvjGXjqWiI1R5k-Xror3R5CnRQXhfZAQ&confirm=t&uuid=89cc1513-464e-44d6-b9ec-9a071a4af39e
To: /content/file.zip
100%|██████████| 187M/187M [00:06<00:00, 27.2MB/s]


In [35]:
#@title 데이터 test
#https://drive.google.com/file/d/1l3c7AI5nl-Bru8NUmHZuGWsToqpKwQtr/view?usp=drive_link

import gdown, zipfile, os

file_id = '1l3c7AI5nl-Bru8NUmHZuGWsToqpKwQtr'
output = 'file.zip'

gdown.download(f'https://drive.google.com/uc?id={file_id}', output, quiet=False)

output_dir = 'test_data'
os.makedirs(output_dir, exist_ok=True)

with zipfile.ZipFile(output, 'r') as z:
  z.extractall(output_dir)

Downloading...
From: https://drive.google.com/uc?id=1l3c7AI5nl-Bru8NUmHZuGWsToqpKwQtr
To: /content/file.zip
100%|██████████| 15.0M/15.0M [00:00<00:00, 65.2MB/s]


In [53]:
def load_data(base_dir='cloud_2'):
  images_path = []
  labels = []
  class_names = []

  for class_name in os.listdir(base_dir):
    class_name_path = os.path.join(base_dir, class_name)

    # 디렉토리인 경우만
    if os.path.isdir(class_name_path):
      for image_name in os.listdir(class_name_path):
        image_path = os.path.join(class_name_path, image_name)
        images_path.append(image_path)
        labels.append(class_name)

  return np.array(images_path), np.array(labels)

images_path, labels = load_data()
print(images_path.shape, labels.shape)
print(images_path[:5], labels[:5])
type(images_path), type(labels)

train_aug_images_path=images_path
train_aug_labels=labels
train_aug_images_path.shape, train_aug_labels.shape

(12846,) (12846,)
['cloud_2/Ns/aug_2_Ns-N058.jpg' 'cloud_2/Ns/aug_2_Ns-N091.jpg'
 'cloud_2/Ns/aug_3_Ns-N019.jpg' 'cloud_2/Ns/aug_1_Ns-N172.jpg'
 'cloud_2/Ns/aug_4_Ns-N120.jpg'] ['Ns' 'Ns' 'Ns' 'Ns' 'Ns']


((12846,), (12846,))

In [45]:
rm -rf cloud_2/St

In [54]:
# @title 메타데이터 생성 (val)
from sklearn.model_selection import train_test_split

def load_data(base_dir='test_data/241127_cloud_img'):
  val_aug_images = []
  val_aug_labels = []
  class_names = []

  train_base = os.path.join(base_dir,"sample_cloud_img")

  for class_name in os.listdir(train_base):
    class_name_path = os.path.join(train_base, class_name)

    # 디렉토리인 경우만
    if os.path.isdir(class_name_path):
      for image_name in os.listdir(class_name_path):
        image_path = os.path.join(class_name_path, image_name)
        val_aug_images.append(image_path)
        val_aug_labels.append(class_name)

  return np.array(val_aug_images), np.array(val_aug_labels)

val_aug_images_path, val_aug_labels = load_data()
print(val_aug_images_path.shape, val_aug_labels.shape)
print(val_aug_images_path, val_aug_labels)
type(val_aug_images_path), type(val_aug_labels)

(35,) (35,)
['test_data/241127_cloud_img/sample_cloud_img/Ns/Ns_5.jpeg'
 'test_data/241127_cloud_img/sample_cloud_img/Ns/Ns_3.jpg'
 'test_data/241127_cloud_img/sample_cloud_img/Ns/Ns_1.jpg'
 'test_data/241127_cloud_img/sample_cloud_img/Ns/Ns_4.jpeg'
 'test_data/241127_cloud_img/sample_cloud_img/Ns/Ns_2.jpg'
 'test_data/241127_cloud_img/sample_cloud_img/Sc/Sc_3.jpg'
 'test_data/241127_cloud_img/sample_cloud_img/Sc/다운로드 (1).jpeg'
 'test_data/241127_cloud_img/sample_cloud_img/Sc/Sc_1.jpg'
 'test_data/241127_cloud_img/sample_cloud_img/Sc/Sc_2.jpg'
 'test_data/241127_cloud_img/sample_cloud_img/Sc/다운로드.jpeg'
 'test_data/241127_cloud_img/sample_cloud_img/Sc/다운로드 (2).jpeg'
 'test_data/241127_cloud_img/sample_cloud_img/Ci/Ci_1.jpg'
 'test_data/241127_cloud_img/sample_cloud_img/Ci/Ci_2.jpg'
 'test_data/241127_cloud_img/sample_cloud_img/Ci/Ci_3.jpg'
 'test_data/241127_cloud_img/sample_cloud_img/Cb/Cb_2.jpg'
 'test_data/241127_cloud_img/sample_cloud_img/Cb/Cb_3.jpg'
 'test_data/241127_cloud_img/sa

(numpy.ndarray, numpy.ndarray)

In [55]:
# @title 메타데이터 생성 (test)
from sklearn.model_selection import train_test_split

def load_data(base_dir='test_data/241127_cloud_img'):
  val_aug_images = []
  val_aug_labels = []
  class_names = []

  train_base = os.path.join(base_dir,"test_cloud_img")

  for class_name in os.listdir(train_base):
    class_name_path = os.path.join(train_base, class_name)

    # 디렉토리인 경우만
    if os.path.isdir(class_name_path):
      for image_name in os.listdir(class_name_path):
        image_path = os.path.join(class_name_path, image_name)
        val_aug_images.append(image_path)
        val_aug_labels.append(class_name)

  return np.array(val_aug_images), np.array(val_aug_labels)

test_aug_images_path,test_aug_labels = load_data()
print(test_aug_images_path.shape, test_aug_labels.shape)
print(test_aug_images_path, test_aug_labels)
type(test_aug_images_path), type(test_aug_labels)

(37,) (37,)
['test_data/241127_cloud_img/test_cloud_img/Ns/Ns_3.jpg'
 'test_data/241127_cloud_img/test_cloud_img/Ns/다운로드 (1).jpeg'
 'test_data/241127_cloud_img/test_cloud_img/Ns/Ns_1.jpg'
 'test_data/241127_cloud_img/test_cloud_img/Ns/다운로드.jpeg'
 'test_data/241127_cloud_img/test_cloud_img/Ns/Ns_2.jpg'
 'test_data/241127_cloud_img/test_cloud_img/Ns/다운로드 (2).jpeg'
 'test_data/241127_cloud_img/test_cloud_img/Sc/Sc_5.jpg'
 'test_data/241127_cloud_img/test_cloud_img/Sc/Sc_6.jpg'
 'test_data/241127_cloud_img/test_cloud_img/Sc/Sc_3.jpg'
 'test_data/241127_cloud_img/test_cloud_img/Sc/Sc_4.jpg'
 'test_data/241127_cloud_img/test_cloud_img/Sc/Sc_1.jpg'
 'test_data/241127_cloud_img/test_cloud_img/Sc/Sc_2.jpg'
 'test_data/241127_cloud_img/test_cloud_img/Ci/Ci_1.jpg'
 'test_data/241127_cloud_img/test_cloud_img/Ci/Ci_2.jpg'
 'test_data/241127_cloud_img/test_cloud_img/Ci/Ci_3.jpg'
 'test_data/241127_cloud_img/test_cloud_img/Cb/Cb_2.jpg'
 'test_data/241127_cloud_img/test_cloud_img/Cb/Cb_3.jpg'
 'test_d

(numpy.ndarray, numpy.ndarray)

In [56]:
# @title CloudSequence 생성
class CloudSequence(Sequence):
    def __init__(self, images, labels, batch_size=32,
                 target_size=(224, 224), preprocess_function=None):
        self.images = images
        self.labels = labels
        self.batch_size = batch_size
        self.target_size = target_size
        self.preprocess_function = preprocess_function  # 스케일링 함수

    def __len__(self):
        # 데이터 수 = 원본 이미지 수 × 증강 횟수
        return int(np.ceil(self.images.shape[0]/ self.batch_size))

    def __getitem__(self, index):
        start = index * self.batch_size
        stop = (index + 1) * self.batch_size

        # 증강을 반복하여 데이터 생성
        batch_images = []
        batch_labels = []
        for img_path, label in zip(self.images[start:stop], self.labels[start:stop]):
            image = cv2.imread(img_path)
            image = cv2.resize(image, self.target_size)
            batch_images.append(image)
            batch_labels.append(label)

        # 최종 배치 반환
        batch_images = np.array(batch_images)
        batch_labels = np.array(batch_labels)
        return np.array(batch_images), np.array(batch_labels)


In [57]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
train_aug_labels = label_encoder.fit_transform(train_aug_labels)
train_aug_labels = np.array(train_aug_labels, dtype='int32')
print(np.unique(train_aug_labels))
print(train_aug_labels.dtype)

val_aug_labels = label_encoder.transform(val_aug_labels)
val_aug_labels = np.array(val_aug_labels, dtype='int32')
print(np.unique(val_aug_labels))
print(val_aug_labels.dtype)

[0 1 2 3 4 5 6 7 8 9]
int32
[0 1 2 3 4 5 6 7 8 9]
int32


In [58]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
tr_seq=CloudSequence(train_aug_images_path, train_aug_labels, preprocess_function=preprocess_input)
val_seq=CloudSequence(val_aug_images_path, val_aug_labels, preprocess_function=preprocess_input)
test_seq=CloudSequence(test_aug_images_path, test_aug_labels, preprocess_function=preprocess_input)

In [18]:
tr_seq.__getitem__(0)[0].shape

(32, 224, 224, 3)

In [59]:
# @title 모델 compile
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=Adam(learning_rate=0.001), # 'adam' -> Adam() 실제 객체로 변환후 학습이 진행되고 있다?
    metrics=['accuracy']
)

early_stopping_cb = EarlyStopping(patience=6, verbose=1, restore_best_weights=True)
reduce_lr_on_plateau_cb = ReduceLROnPlateau(patience=3, factor=0.5, verbose=1)

In [60]:
# 학습
history = model.fit(
    tr_seq,
    epochs=30,
    batch_size=32,
    validation_data=val_seq,
    callbacks=[early_stopping_cb, reduce_lr_on_plateau_cb])

Epoch 1/30


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


402/402 ━━━━━━━━━━━━━━━━━━━━ 121s 192ms/step - accuracy: 0.1406 - loss: 2.9086 - val_accuracy: 0.1714 - val_loss: 2.2929 - learning_rate: 0.0010
Epoch 2/30
402/402 ━━━━━━━━━━━━━━━━━━━━ 41s 102ms/step - accuracy: 0.1586 - loss: 2.2479 - val_accuracy: 0.1714 - val_loss: 2.3831 - learning_rate: 0.0010
Epoch 3/30
402/402 ━━━━━━━━━━━━━━━━━━━━ 41s 103ms/step - accuracy: 0.2217 - loss: 2.0881 - val_accuracy: 0.1714 - val_loss: 3.0408 - learning_rate: 0.0010
Epoch 4/30
402/402 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step - accuracy: 0.2426 - loss: 1.9688
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
402/402 ━━━━━━━━━━━━━━━━━━━━ 41s 103ms/step - accuracy: 0.2426 - loss: 1.9688 - val_accuracy: 0.0857 - val_loss: 3.2578 - learning_rate: 0.0010
Epoch 5/30
402/402 ━━━━━━━━━━━━━━━━━━━━ 82s 102ms/step - accuracy: 0.2724 - loss: 1.7957 - val_accuracy: 0.1714 - val_loss: 3.8894 - learning_rate: 5.0000e-04
Epoch 6/30
402/402 ━━━━━━━━━━━━━━━━━━━━ 42s 103ms/step - accuracy: 0.3011 - lo

In [69]:
from keras.models import Model

# 모든 레이어의 출력을 반환하는 모델 생성
layer_outputs = [layer.output for layer in model.layers]  # 레이어 출력 리스트
activation_model = Model(inputs=model.input, outputs=layer_outputs)

b=next(iter(test_seq))
img=b[0]
# # 입력 이미지를 모델에 전달하여 모든 레이어 출력 얻기
activations = activation_model.predict(img)

# 각 레이어 출력 시각화
import matplotlib.pyplot as plt

for i, activation in enumerate(activations):
    print(f"Layer {i + 1} - {model.layers[i].name}: {activation.shape}")
    if len(activation.shape) == 4:  # Conv 레이어의 경우
        n_features = activation.shape[-1]
        size = activation.shape[1]
        # 첫 번째 필터의 맵 시각화 (모든 필터를 보고 싶으면 루프 추가)
        plt.matshow(activation[0, :, :, 0], cmap='viridis')
        plt.title(f"Layer {i + 1}: {model.layers[i].name}")
#         plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [71]:
for i, activation in enumerate(activations):
    mean_activation = activation.mean()  # 평균 활성화 값
    print(f"Layer {i + 1}: Mean Activation = {mean_activation}")

Layer 1: Mean Activation = 132.01568603515625
Layer 2: Mean Activation = -0.6772018074989319
Layer 3: Mean Activation = -0.09889751672744751
Layer 4: Mean Activation = 1.1863921880722046
Layer 5: Mean Activation = -0.1464354246854782
Layer 6: Mean Activation = -0.15630684792995453
Layer 7: Mean Activation = 0.09752464294433594
Layer 8: Mean Activation = 0.05406349524855614
Layer 9: Mean Activation = 0.06520494073629379
Layer 10: Mean Activation = 0.019842155277729034
Layer 11: Mean Activation = 0.02149592712521553
Layer 12: Mean Activation = 0.08408234268426895
Layer 13: Mean Activation = 0.08260076493024826
Layer 14: Mean Activation = -0.001976636704057455
Layer 15: Mean Activation = -0.0018850179621949792
Layer 16: Mean Activation = 0.007348085753619671
Layer 17: Mean Activation = 0.0002636730787344277
Layer 18: Mean Activation = -0.012071621604263783
Layer 19: Mean Activation = 0.0016227884916588664
Layer 20: Mean Activation = 0.002013216493651271
Layer 21: Mean Activation = 0.02898

In [76]:
from keras.models import Model
import numpy as np

last_conv_layer = model.get_layer('conv2d_3')  # 마지막 conv 레이어
grads = K.gradients(model.output[:, class_index], last_conv_layer.output)[0]
pooled_grads = K.mean(grads, axis=(0, 1, 2))

iterate = K.function([model.input], [pooled_grads, last_conv_layer.output[0]])
pooled_grads_value, conv_layer_output_value = iterate([img])

for i in range(len(pooled_grads_value)):
    conv_layer_output_value[:, :, i] *= pooled_grads_value[i]

heatmap = np.mean(conv_layer_output_value, axis=-1)

ValueError: No such layer: conv2d_3. Existing layers are: ['input_layer_2', 'Conv1', 'bn_Conv1', 'Conv1_relu', 'expanded_conv_depthwise', 'expanded_conv_depthwise_BN', 'expanded_conv_depthwise_relu', 'expanded_conv_project', 'expanded_conv_project_BN', 'block_1_expand', 'block_1_expand_BN', 'block_1_expand_relu', 'block_1_pad', 'block_1_depthwise', 'block_1_depthwise_BN', 'block_1_depthwise_relu', 'block_1_project', 'block_1_project_BN', 'block_2_expand', 'block_2_expand_BN', 'block_2_expand_relu', 'block_2_depthwise', 'block_2_depthwise_BN', 'block_2_depthwise_relu', 'block_2_project', 'block_2_project_BN', 'block_2_add', 'block_3_expand', 'block_3_expand_BN', 'block_3_expand_relu', 'block_3_pad', 'block_3_depthwise', 'block_3_depthwise_BN', 'block_3_depthwise_relu', 'block_3_project', 'block_3_project_BN', 'block_4_expand', 'block_4_expand_BN', 'block_4_expand_relu', 'block_4_depthwise', 'block_4_depthwise_BN', 'block_4_depthwise_relu', 'block_4_project', 'block_4_project_BN', 'block

In [ ]:

# 학습결과 시각화
pd.DataFrame(history.history).plot()
plt.show()

# 평가
loss, accuracy = model.evaluate(test_aug_images_path, test_aug_labels)
print(f'loss: {loss:.4f}, accuracy: {accuracy:.4f}')


In [ ]:
"https://drive.google.com/file/d/1ZKIFRjaxSAssTO36STrG0XrHurUqfK1B/view?usp=sharing"

import gdown, zipfile, os

file_id = '1ZKIFRjaxSAssTO36STrG0XrHurUqfK1B'
output = 'file.zip'

gdown.download(f'https://drive.google.com/uc?id={file_id}', output, quiet=False)

output_dir = 'cloud'
os.makedirs(output_dir, exist_ok=True)

with zipfile.ZipFile(output, 'r') as z:
  z.extractall(output_dir)

In [ ]:
# @title 모델 예측

pred_proba = model.predict(test_aug_images_path)
print(pred_proba.shape)

class_names = np.array(['Ac', 'As', 'Cb', 'Cc', 'Ci', 'Cs', 'Ct', 'Cu', 'Ns', 'Sc', 'St'])

def show_images(images, labels, ncols = 8):
  fig, ax = plt.subplots(nrows = 1, ncols = ncols, figsize = (20, 6))
  for i in range(ncols):
    ax[i].imshow(images[i])  # ndarray 전달해서 이미지 출력
    class_name = class_names[labels[i]]
    ax[i].set_title(class_name)

  plt.show()

pred = np.argmax(pred_proba, axis = 1)
for i in range(10):
  print(f'{i} : {class_names[pred[i]]}')

show_images(test_aug_images_path[:10], test_aug_labels[:10])
show_images(test_aug_images_path[:10], pred[:10])

In [ ]:
model.save('best_cloud_mobilenet_2.keras')